In [1]:
import time
import pandas as pd
from zomatopy import Zomato
#get_categories
#get_city_ID
#get_city_name
#get_collections
#get_cuisines
#get_establishment_types
#get_nearby_restaurants
#get_restaurant
#restaurant_search

zconfig1 = {"user_key":"2dde89df168a9311ecd9f1459126144c"} # Stef
zconfig2 = {"user_key":"312a1f0d7bb1f8cbfc611842ca184e1c"} # Kin
zconfig3 = {"user_key":"508a65203e6a0e2a3f87a1720682befa"} # Nandini
zconfig4 = {"user_key":"82f088ef3f4b0e06cf38d0701f38b42f"} # Sophia

### READ DATA

In [2]:
zcities_df = pd.read_csv('../data/kin_nov2/zcities_cleaned.csv') # cleaned zomato cities
cstates_df = pd.read_csv('../data/kin_nov2/top_population_state.csv') # census top 10 migrant populations
cuisines_df = pd.read_csv('../data/kin_nov2/cuisine_ids.csv') # merged cuisines and populations

In [3]:
zcities_df.drop('Unnamed: 0', axis=1, inplace=True)
zcities_df.head(5)

,zcity_id,city_query,state_code
0,276,Dallas TX,TX
1,277,Houston TX,TX
2,278,Austin TX,TX
3,279,Seattle WA,WA
4,280,New York NY,NY


In [4]:
cstates_df.head(5)

,state,state_code,population1,population2,population3,population4,population5,population6,population7,population8,population9,population10
0,ALABAMA,AL,Mexico,Guatemala,Korea,India,China,Germany,Vietnam,Philippines,Honduras,Canada
1,ALASKA,AK,Philippines,Mexico,Korea,Thailand,Canada,Russia,Germany,Dominican Republic,"Oceania, nec",Laos
2,ARIZONA,AZ,Mexico,Canada,India,Philippines,China,Vietnam,Germany,Guatemala,Korea,Iraq
3,ARKANSAS,AR,Mexico,El Salvador,India,"Oceania, nec",Guatemala,Vietnam,China,Philippines,Laos,Germany
4,CALIFORNIA,CA,Mexico,Philippines,China,Vietnam,India,El Salvador,Korea,Guatemala,Iran,Taiwan


In [5]:
cuisines_df.head(5)

,population,cuisine_names,cuisine_id
0,Mexico,"Mexico, Taco, Tex-Mex","73,997,150"
1,Canada,Canada,381
2,India,India,148
3,China,"China, Sichuan","25,128"
4,Vietnam,Vietnam,99


### QUERY DATAFRAME

In [6]:
query_df = zcities_df.copy()
query_df = query_df.merge(cstates_df, on='state_code', how='left')
perc = query_df.population1.isna().sum()/len(query_df)
print(f'missing state merge percentage {perc}')

missing state merge percentage 0.0


In [7]:
query_df.head(5)

,zcity_id,city_query,state_code,state,population1,population2,population3,population4,population5,population6,population7,population8,population9,population10
0,276,Dallas TX,TX,TEXAS,Mexico,India,El Salvador,Vietnam,Honduras,China,Philippines,Guatemala,Nigeria,Pakistan
1,277,Houston TX,TX,TEXAS,Mexico,India,El Salvador,Vietnam,Honduras,China,Philippines,Guatemala,Nigeria,Pakistan
2,278,Austin TX,TX,TEXAS,Mexico,India,El Salvador,Vietnam,Honduras,China,Philippines,Guatemala,Nigeria,Pakistan
3,279,Seattle WA,WA,WASHINGTON,Mexico,India,Philippines,China,Vietnam,Korea,Canada,Ukraine,Russia,Germany
4,280,New York NY,NY,NEW YORK,Dominican Republic,China,Jamaica,Mexico,Ecuador,Guyana,India,Haiti,El Salvador,Colombia


In [8]:
query_df = pd.wide_to_long(query_df, stubnames='population',
                           i=['zcity_id', 'state_code'], j='top')
query_df = query_df.reset_index()
query_df.head(10)

,zcity_id,state_code,top,city_query,state,population
0,276,TX,1,Dallas TX,TEXAS,Mexico
1,276,TX,2,Dallas TX,TEXAS,India
2,276,TX,3,Dallas TX,TEXAS,El Salvador
3,276,TX,4,Dallas TX,TEXAS,Vietnam
4,276,TX,5,Dallas TX,TEXAS,Honduras
5,276,TX,6,Dallas TX,TEXAS,China
6,276,TX,7,Dallas TX,TEXAS,Philippines
7,276,TX,8,Dallas TX,TEXAS,Guatemala
8,276,TX,9,Dallas TX,TEXAS,Nigeria
9,276,TX,10,Dallas TX,TEXAS,Pakistan


In [9]:
query_df = query_df.reset_index()

In [10]:
query_df = query_df.merge(cuisines_df, on='population', how='left')
perc = query_df.cuisine_id.isna().sum()/len(query_df)
print(f'missing cuisine_id percentage {perc}')
check_missings = query_df[query_df.cuisine_id.isna()].population
check_missings = check_missings.unique()
query_df.head(10)

missing cuisine_id percentage 0.0


,index,zcity_id,state_code,top,city_query,state,population,cuisine_names,cuisine_id
0,0,276,TX,1,Dallas TX,TEXAS,Mexico,"Mexico, Taco, Tex-Mex","73,997,150"
1,1,276,TX,2,Dallas TX,TEXAS,India,India,148
2,2,276,TX,3,Dallas TX,TEXAS,El Salvador,El Salvador,601
3,3,276,TX,4,Dallas TX,TEXAS,Vietnam,Vietnam,99
4,4,276,TX,5,Dallas TX,TEXAS,Honduras,South America,972
5,5,276,TX,6,Dallas TX,TEXAS,China,"China, Sichuan","25,128"
6,5,276,TX,6,Dallas TX,TEXAS,China,"Hong Kong, Dim Sum","121, 411"
7,6,276,TX,7,Dallas TX,TEXAS,Philippines,Philippines,112
8,7,276,TX,8,Dallas TX,TEXAS,Guatemala,Guatemala,"972,136"
9,8,276,TX,9,Dallas TX,TEXAS,Nigeria,Nigeria,152


In [11]:
check_missings

array([], dtype=object)

### Zomato top populations/cuisines per city query

In [17]:
def zomato_city_restaurant_count_query(query_df, zconfig, api_key):
    zomato = Zomato(zconfig)
    response_df_path = f'../results/city_restaurant_count{api_key}.pkl'
    print(response_df_path)
    
    zcity_id_list = []
    cuisine_id_list = []
    json_list = []
    count_list = []
    query_list = []
    for i, row in query_df.iterrows():        
        # Security saves of Zomato calls
        if i>0 and (i % 10 == 0):
            time.sleep(5)
            response_df = pd.DataFrame.from_dict({'zcity_id': zcity_id_list, 
                                                  'cuisines': cuisine_id_list,
                                                  'json': json_list,
                                                  'query': query_list,
                                                  'counts': count_list})
            response_df.to_pickle(response_df_path)
        
        try:
            count, json, query = zomato.get_city_restaurant_count(city_ID=row.zcity_id,
                                                                  cuisines=row.cuisine_id)
            zcity_id_list += [row.zcity_id]
            cuisine_id_list += [row.cuisine_id]
            json_list += [json]
            count_list += [count]
            query_list += [query]
            
            print(f'{i} city: {row.zcity_id} cuisines: {row.cuisine_id} count: {count}')
        except:
            print(f"failed city query {row.city_query}")
            pass
    
    #Final save
    response_df = pd.DataFrame.from_dict({'zcity_id': zcity_id_list, 
                                          'cuisines': cuisine_id_list,
                                          'json': json_list,
                                          'query': query_list,
                                          'counts': count_list})
    response_df.to_pickle(response_df_path)
    return response_df

query_df1 = query_df[:500]
query_df2 = query_df[500:961]
query_df3 = query_df[961:1961]
query_df4 = query_df[1961:2960]
query_df5 = query_df[2960:3960]
query_df6 = query_df[3960:]
#city_restaurants_df = zomato_city_restaurant_count_query(query_df=query_df1, zconfig=zconfig1, api_key=1)
#city_restaurants_df = zomato_city_restaurant_count_query(query_df=query_df2, zconfig=zconfig2, api_key=2)
#city_restaurants_df = zomato_city_restaurant_count_query(query_df=query_df3, zconfig=zconfig3, api_key=3)
city_restaurants_df = zomato_city_restaurant_count_query(query_df=query_df4, zconfig=zconfig4, api_key=4)

../results/city_restaurant_count4.pkl
1961 city: 1168 cuisines: 972 count: 3
1962 city: 1168 cuisines: 25,128 count: 61
1963 city: 1168 cuisines: 121, 411 count: 0
1964 city: 1168 cuisines: 112 count: 2
1965 city: 1168 cuisines: 972,136 count: 451
1966 city: 1168 cuisines: 152 count: 1
1967 city: 1168 cuisines: 139 count: 0
1968 city: 1171 cuisines: 73,997,150 count: 77
1969 city: 1171 cuisines: 148 count: 2
1970 city: 1171 cuisines: 601 count: 0
1971 city: 1171 cuisines: 99 count: 10
1972 city: 1171 cuisines: 972 count: 0
1973 city: 1171 cuisines: 25,128 count: 28
1974 city: 1171 cuisines: 121, 411 count: 0
1975 city: 1171 cuisines: 112 count: 0
1976 city: 1171 cuisines: 972,136 count: 71
1977 city: 1171 cuisines: 152 count: 0
1978 city: 1171 cuisines: 139 count: 0
1979 city: 1174 cuisines: 73,997,150 count: 170
1980 city: 1174 cuisines: 148 count: 2
1981 city: 1174 cuisines: 601 count: 0
1982 city: 1174 cuisines: 99 count: 10
1983 city: 1174 cuisines: 972 count: 0
1984 city: 1174 cui

2160 city: 1228 cuisines: 99 count: 20
2161 city: 1228 cuisines: 25,128 count: 393
2162 city: 1228 cuisines: 121, 411 count: 3
2163 city: 1228 cuisines: 972 count: 10
2164 city: 1228 cuisines: 972,136 count: 222
2165 city: 1228 cuisines: 149 count: 1
2166 city: 1240 cuisines: 73,997,150 count: 114
2167 city: 1240 cuisines: 148 count: 4
2168 city: 1240 cuisines: 112 count: 0
2169 city: 1240 cuisines: 25,128 count: 47
2170 city: 1240 cuisines: 121, 411 count: 1
2171 city: 1240 cuisines: 99 count: 12
2172 city: 1240 cuisines: 67 count: 10
2173 city: 1240 cuisines: 381 count: 0
2174 city: 1240 cuisines: 451 count: 0
2175 city: 1240 cuisines: 84 count: 0
2176 city: 1240 cuisines: 134 count: 3
2177 city: 1246 cuisines: 73,997,150 count: 64
2178 city: 1246 cuisines: 148 count: 1
2179 city: 1246 cuisines: 112 count: 0
2180 city: 1246 cuisines: 25,128 count: 24
2181 city: 1246 cuisines: 121, 411 count: 1
2182 city: 1246 cuisines: 99 count: 2
2183 city: 1246 cuisines: 67 count: 2
2184 city: 1246

2360 city: 3816 cuisines: 121, 411 count: 175
2361 city: 3816 cuisines: 67 count: 449
2362 city: 3816 cuisines: 162 count: 223
2363 city: 3816 cuisines: 601 count: 67
2364 city: 3832 cuisines: 148 count: 819
2365 city: 3832 cuisines: 958 count: 214
2366 city: 3832 cuisines: 73,997,150 count: 1916
2367 city: 3832 cuisines: 112 count: 77
2368 city: 3832 cuisines: 287 count: 184
2369 city: 3832 cuisines: 316 count: 71
2370 city: 3832 cuisines: 25,128 count: 4089
2371 city: 3832 cuisines: 121, 411 count: 176
2372 city: 3832 cuisines: 67 count: 453
2373 city: 3832 cuisines: 162 count: 227
2374 city: 3832 cuisines: 601 count: 67
2375 city: 3895 cuisines: 148 count: 856
2376 city: 3895 cuisines: 958 count: 215
2377 city: 3895 cuisines: 73,997,150 count: 1956
2378 city: 3895 cuisines: 112 count: 83
2379 city: 3895 cuisines: 287 count: 184
2380 city: 3895 cuisines: 316 count: 71
2381 city: 3895 cuisines: 25,128 count: 4203
2382 city: 3895 cuisines: 121, 411 count: 181
2383 city: 3895 cuisines: 

2558 city: 5546 cuisines: 381 count: 0
2559 city: 5546 cuisines: 972,136 count: 2156
2560 city: 5554 cuisines: 73,997,150 count: 2010
2561 city: 5554 cuisines: 148 count: 220
2562 city: 5554 cuisines: 25,128 count: 1021
2563 city: 5554 cuisines: 121, 411 count: 35
2564 city: 5554 cuisines: 22 count: 0
2565 city: 5554 cuisines: 112 count: 33
2566 city: 5554 cuisines: 601 count: 5
2567 city: 5554 cuisines: 67 count: 128
2568 city: 5554 cuisines: 134 count: 31
2569 city: 5554 cuisines: 381 count: 0
2570 city: 5554 cuisines: 972,136 count: 2166
2571 city: 5649 cuisines: 73,997,150 count: 70
2572 city: 5649 cuisines: 148 count: 6
2573 city: 5649 cuisines: 99 count: 4
2574 city: 5649 cuisines: 25,128 count: 34
2575 city: 5649 cuisines: 121, 411 count: 0
2576 city: 5649 cuisines: 972,136 count: 70
2577 city: 5649 cuisines: 112 count: 0
2578 city: 5649 cuisines: 601 count: 0
2579 city: 5649 cuisines: 67 count: 5
2580 city: 5649 cuisines: 134 count: 0
2581 city: 5649 cuisines: 152 count: 0
2582

2759 city: 8181 cuisines: 99 count: 0
2760 city: 8181 cuisines: 148 count: 1
2761 city: 8181 cuisines: 972,136 count: 13
2762 city: 8181 cuisines: 25,128 count: 4
2763 city: 8181 cuisines: 121, 411 count: 0
2764 city: 8181 cuisines: 112 count: 0
2765 city: 8181 cuisines: 67 count: 1
2766 city: 8181 cuisines: 134 count: 1
2767 city: 8181 cuisines: 972 count: 0
2768 city: 8181 cuisines: 381 count: 0
2769 city: 8279 cuisines: 73,997,150 count: 21
2770 city: 8279 cuisines: 99 count: 0
2771 city: 8279 cuisines: 148 count: 1
2772 city: 8279 cuisines: 972,136 count: 20
2773 city: 8279 cuisines: 25,128 count: 7
2774 city: 8279 cuisines: 121, 411 count: 0
2775 city: 8279 cuisines: 112 count: 0
2776 city: 8279 cuisines: 67 count: 0
2777 city: 8279 cuisines: 134 count: 0
2778 city: 8279 cuisines: 972 count: 1
2779 city: 8279 cuisines: 381 count: 0
2780 city: 8340 cuisines: 73,997,150 count: 172
2781 city: 8340 cuisines: 99 count: 12
2782 city: 8340 cuisines: 25,128 count: 62
2783 city: 8340 cuisi

failed city query Beaverton OR


In [13]:
query_df2 = query_df[500:961]

In [14]:
query_df2

,index,zcity_id,state_code,top,city_query,state,population,cuisine_names,cuisine_id
500,456,523,CA,7,Victorville CA,CALIFORNIA,Korea,Korea,67
501,457,523,CA,8,Victorville CA,CALIFORNIA,Guatemala,Guatemala,"972,136"
502,458,523,CA,9,Victorville CA,CALIFORNIA,Iran,Iran,140
503,459,523,CA,10,Victorville CA,CALIFORNIA,Taiwan,Taiwan,190
504,460,526,CA,1,Santa Barbara CA,CALIFORNIA,Mexico,"Mexico, Taco, Tex-Mex","73,997,150"
...,...,...,...,...,...,...,...,...,...
956,880,712,IN,1,Muncie IN,INDIANA,Mexico,"Mexico, Taco, Tex-Mex","73,997,150"
957,881,712,IN,2,Muncie IN,INDIANA,India,India,148
958,882,712,IN,3,Muncie IN,INDIANA,China,"China, Sichuan","25,128"
959,882,712,IN,3,Muncie IN,INDIANA,China,"Hong Kong, Dim Sum","121, 411"


In [26]:
ver = pd.read_pickle('../results/city_restaurant_count1.pkl')

In [27]:
ver

,zcity_id,cuisines,json,query,counts
0,276,"73,997,150","{'results_found': 1825, 'results_start': 0, 'r...",https://developers.zomato.com/api/v2.1/search?...,1825
1,276,148,"{'results_found': 180, 'results_start': 0, 're...",https://developers.zomato.com/api/v2.1/search?...,180
2,276,601,"{'results_found': 29, 'results_start': 0, 'res...",https://developers.zomato.com/api/v2.1/search?...,29
3,276,99,"{'results_found': 176, 'results_start': 0, 're...",https://developers.zomato.com/api/v2.1/search?...,176
4,276,972,"{'results_found': 35, 'results_start': 0, 'res...",https://developers.zomato.com/api/v2.1/search?...,35
...,...,...,...,...,...
495,523,"25,128","{'results_found': 21, 'results_start': 0, 'res...",https://developers.zomato.com/api/v2.1/search?...,21
496,523,"121, 411","{'results_found': 1, 'results_start': 0, 'resu...",https://developers.zomato.com/api/v2.1/search?...,1
497,523,99,"{'results_found': 2, 'results_start': 0, 'resu...",https://developers.zomato.com/api/v2.1/search?...,2
498,523,148,"{'results_found': 2, 'results_start': 0, 'resu...",https://developers.zomato.com/api/v2.1/search?...,2
